In [28]:
import ipynb
import pandas as pd
import numpy as np
import requests
import api_key
import json
from tqdm import tqdm
from geopy import distance


from ipynb.fs.full.mmtc_scrape import *

In [29]:
#clean companies df: no cleaning needed
df_companies

,Name,Phone,Email,Authorization Status,License Number
0,Ayr Cannabis Dispensary,833-254-4877,Info@libertyhealthsciences.com,Dispensing Authorization,MMTC-2015-0002
1,Cannabist,800-714-9215,info@col-carefl.com,Dispensing Authorization,MMTC-2017-0011
2,"Cookies Florida, Inc.",NaN,NaN,Dispensing Authorization,MMTC-2019-0018
3,Curaleaf,877-303-0741,info.fl@curaleaf.com,Dispensing Authorization,MMTC-2015-0001
4,FLUENT,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003
5,Gold Leaf,NaN,NaN,Dispensing Authorization,MMTC-2019-0019
6,Green Dragon,720-600-9555,support@greendragon.com,Dispensing Authorization,MMTC-2019-0021
7,GrowHealthy,863-223-8882,info@GrowHealthy.com,Dispensing Authorization,MMTC-2016-0007
8,GTI (Rise Dispensaries),305-306-8772,FLinfo@gtigrows.com,Dispensing Authorization,MMTC-2017-0013
9,House of Platinum Cannabis,NaN,NaN,Dispensing Authorization,MMTC-2018-0014


In [30]:
df_locations.count

<bound method DataFrame.count of                      COMPANY                         ADDRESS EMAIL ADDRESS  \
0    Ayr Cannabis Dispensary     6325 N Orange Blossom Trail             -   
1    Ayr Cannabis Dispensary               7390 Aloma Avenue             -   
2    Ayr Cannabis Dispensary            4650 N Alafaya Trail             -   
3    Ayr Cannabis Dispensary             440 N State Road 19             -   
4    Ayr Cannabis Dispensary  6930 Cypress Gardens Boulevard             -   
..                       ...                             ...           ...   
584                 VidaCann        2007 W Kennedy Boulevard             -   
585                 VidaCann              5203 Cortez Road W             -   
586                 VidaCann           1101 S Powerline Road             -   
587                 VidaCann          1663 Georgia Street NE             -   
588                 VidaCann                1027 N Nova Road             -   

    PHONE             CITY ZIP

In [31]:
#clean locations df

#drop empty columns
df_locations = df_locations.drop(['EMAIL ADDRESS', 'PHONE'], axis=1, errors='ignore')

#modify company names to matche the df_companies dataframe
df_locations.replace('MüV', 'MuV', inplace=True)
df_locations.replace('Sunnyside*', 'Sunnyside', inplace=True)

#left join companies onto locations on company name
merged_df = df_locations.merge(right=df_companies, how='left', left_on='COMPANY', right_on='Name')

# clean merged_df
merged_df = merged_df.drop('Name', axis=1)

#provide a detailed request string for geoapify
merged_df['Full Address'] = merged_df['ADDRESS'] + ", " + merged_df['CITY'] + ', FL ' + merged_df['ZIP CODE']

#get only the FLUENT Dispensaries
fluent_dispensaries = merged_df.loc[merged_df['COMPANY'] == 'FLUENT']


fluent_dispensaries.head()
merged_df.count()

COMPANY                 589
ADDRESS                 589
CITY                    589
ZIP CODE                589
COUNTY                  589
Phone                   545
Email                   545
Authorization Status    577
License Number          577
Full Address            589
dtype: int64

In [32]:
# miami_df_test = merged_df.loc[merged_df['CITY'].str.contains("Miami")]
# miami_df_test.count()

In [33]:
#summary analysis: count dispensaries by MMTC
merged_df['COMPANY'].value_counts()


Trulieve                      126
MuV                            70
Ayr Cannabis Dispensary        62
Curaleaf                       60
Surterra Wellness              45
FLUENT                         33
Green Dragon                   33
Sunnyside                      33
VidaCann                       26
Sanctuary Cannabis             19
GrowHealthy                    18
Cannabist                      14
Sunburn                        11
GTI (Rise Dispensaries)         9
Insa                            9
Jungle Boys                     8
The Flowery                     5
House of Platinum Cannabis      4
Cookies                         3
Gold Leaf                       1
Name: COMPANY, dtype: int64

In [34]:
example_df = fluent_dispensaries.loc[fluent_dispensaries['ADDRESS'].str.contains('175 NW')]
example_df

,COMPANY,ADDRESS,CITY,ZIP CODE,COUNTY,Phone,Email,Authorization Status,License Number,Full Address
164,FLUENT,175 NW 167th Street,North Miami Beach,33169,Miami-Dade,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"175 NW 167th Street, North Miami Beach, FL 33169"


In [35]:
#input raw location dataset
input_ = merged_df

# #provide a detailed request string for geoapify
# input_['Full Address'] = input_['ADDRESS'] + ", " + input_['CITY'] + ', FL ' + input_['ZIP CODE']
# # input_ = input_[['ADDRESS']]


#clean input dataset
input_['Full Address'] = input_['Full Address'].str.replace('11245 SW 211 Street, Miami, FL 33189','11245 SW 211th St, Miami, FL  33189, United States')


#define output dataframe
output = pd.DataFrame(columns=['ADDRESS','Latitude', 'Longitude'])

#define failed list
failed_df = pd.DataFrame(columns=['ADDRESS'])
input_.head()

#limit the requests to the city of Miami Only
#miami_df = input_.loc[input_['CITY'].str.contains("Miami")]

,COMPANY,ADDRESS,CITY,ZIP CODE,COUNTY,Phone,Email,Authorization Status,License Number,Full Address
0,Ayr Cannabis Dispensary,6325 N Orange Blossom Trail,Orlando,32810,Orange,833-254-4877,Info@libertyhealthsciences.com,Dispensing Authorization,MMTC-2015-0002,"6325 N Orange Blossom Trail, Orlando, FL 32810"
1,Ayr Cannabis Dispensary,7390 Aloma Avenue,Winter Park,32792,Orange,833-254-4877,Info@libertyhealthsciences.com,Dispensing Authorization,MMTC-2015-0002,"7390 Aloma Avenue, Winter Park, FL 32792"
2,Ayr Cannabis Dispensary,4650 N Alafaya Trail,Orlando,32826,Orange,833-254-4877,Info@libertyhealthsciences.com,Dispensing Authorization,MMTC-2015-0002,"4650 N Alafaya Trail, Orlando, FL 32826"
3,Ayr Cannabis Dispensary,440 N State Road 19,Palatka,32177,Putnam,833-254-4877,Info@libertyhealthsciences.com,Dispensing Authorization,MMTC-2015-0002,"440 N State Road 19, Palatka, FL 32177"
4,Ayr Cannabis Dispensary,6930 Cypress Gardens Boulevard,Winter Haven,33884,Polk,833-254-4877,Info@libertyhealthsciences.com,Dispensing Authorization,MMTC-2015-0002,"6930 Cypress Gardens Boulevard, Winter Haven, ..."


In [36]:
#geolocate all addresses (get coordinates)

#for every location address,
for row in input_.iterrows():
        
    #generate geoapify request url
    # location_id = row[1]['Location ID']
    #location_name = row[1]['BT Location Name']
    request_string = row[1]['Full Address']
    query_url = f"https://api.geoapify.com/v1/geocode/search?text={request_string}&format=json&apiKey={api_key.api_key}"
    
    try:
        #get returned data as json. This code hits the API
        geo_data = requests.get(query_url).json()

        #select desired fields to return to dataframe
        lat = geo_data['results'][0]['lat']
        lon = geo_data['results'][0]['lon']
        # county = geo_data['results'][0]['county']
        # popularity = geo_data['results'][0]['rank']['popularity']
        # importance = geo_data['results'][0]['rank']['importance']

        #append the data to the results dataframe
        current_frame = pd.DataFrame({'ADDRESS':[request_string],'Latitude':[lat], 'Longitude':[lon]})
        frames = [output, current_frame]
        output = pd.concat(frames)
        
        print('Success')
    
    except:
        current_frame = pd.DataFrame({'ADDRESS':[request_string]})
        failed_df = pd.concat([failed_df, current_frame])
        
        print('Failed')
    
#print the head of the dataframe to ensure that data was successfully loaded
output.head()

#print the failed dataframes
failed_df

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


,ADDRESS


In [37]:
output.count()

merged_df_with_coordinates = input_.merge(right=output, right_on='ADDRESS', left_on='Full Address', how='left')

# clean columns
merged_df_with_coordinates = merged_df_with_coordinates.drop('ADDRESS_y', axis=1)
merged_df_with_coordinates['ADDRESS'] = merged_df_with_coordinates['ADDRESS_x']
merged_df_with_coordinates = merged_df_with_coordinates.drop('ADDRESS_x', axis=1)
merged_df_with_coordinates = merged_df_with_coordinates[['COMPANY', 'ADDRESS', 'CITY', 'ZIP CODE', 'COUNTY', 'Phone', 'Email',
       'Authorization Status', 'License Number', 'Full Address', 'Latitude',
       'Longitude']]

merged_df_with_coordinates.count()
merged_df_with_coordinates.to_csv('./locations_with_coordinates.csv')

#get the fluent dispensaries in miami
fluent_dispensaries = merged_df_with_coordinates.loc[merged_df_with_coordinates['COMPANY'] == 'FLUENT']
fluent_dispensaries.head()

# merged_df_with_coordinates.dtypes
#merged_df_with_coordinates.columns

,COMPANY,ADDRESS,CITY,ZIP CODE,COUNTY,Phone,Email,Authorization Status,License Number,Full Address,Latitude,Longitude
139,FLUENT,971 Cassat Avenue,Jacksonville,32205,Duval,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"971 Cassat Avenue, Jacksonville, FL 32205",30.309614,-81.730240
140,FLUENT,2385 S Ferdon Boulevard,Crestview,32536,Okaloosa,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"2385 S Ferdon Boulevard, Crestview, FL 32536",30.734197,-86.566244
141,FLUENT,500 E Chase Street,Pensacola,32502,Escambia,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"500 E Chase Street, Pensacola, FL 32502",30.416350,-87.203878
142,FLUENT,763 Atlantic Boulevard,Atlantic Beach,32233,Duval,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"763 Atlantic Boulevard, Atlantic Beach, FL 32233",30.324805,-81.408720
143,FLUENT,301 E Nine Road,Pensacola,32514,Escambia,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"301 E Nine Road, Pensacola, FL 32514",30.532395,-87.262019


In [47]:
# shortcut

# use csv to import coordinate data instead of api to save on credits

#merged_df_with_coordinates.read_csv('./locations_with_coordinates.csv')

In [39]:

#run through each fluent dispensary, caclulate the travel time / distance to every other dispensary

def calculate_distance_to_another_dispensary(from_waypoint, to_waypoint):

    #Test case coordinates
    # from_waypoint = [25.928299,-80.203588]
    # to_waypoint = [25.571142,-80.369250]

    #design request query
    url = f"https://api.geoapify.com/v1/routing?waypoints={from_waypoint[0]},{from_waypoint[1]}|{to_waypoint[0]},{to_waypoint[1]}&mode=drive&details=instruction_details&apiKey={api_key.api_key}"

    try:
        result_ = requests.get(url).json()
        distance_in_meters = result_['features'][0]['properties']['distance']
        print('Success')
        
        #convert distance from meters to miles
        distance_in_miles = distance_in_meters * 0.0006213712

        return(distance_in_miles)

    except:

        print(f'Faliure: {url}')



# calculate_distance_to_another_dispensary()




In [40]:
def find_distance(from_waypoint, to_waypoint):

    

    return(distance.distance(from_waypoint, to_waypoint).miles)

In [41]:
fluent_dispensaries

,COMPANY,ADDRESS,CITY,ZIP CODE,COUNTY,Phone,Email,Authorization Status,License Number,Full Address,Latitude,Longitude
139,FLUENT,971 Cassat Avenue,Jacksonville,32205,Duval,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"971 Cassat Avenue, Jacksonville, FL 32205",30.309614,-81.730240
140,FLUENT,2385 S Ferdon Boulevard,Crestview,32536,Okaloosa,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"2385 S Ferdon Boulevard, Crestview, FL 32536",30.734197,-86.566244
141,FLUENT,500 E Chase Street,Pensacola,32502,Escambia,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"500 E Chase Street, Pensacola, FL 32502",30.416350,-87.203878
142,FLUENT,763 Atlantic Boulevard,Atlantic Beach,32233,Duval,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"763 Atlantic Boulevard, Atlantic Beach, FL 32233",30.324805,-81.408720
143,FLUENT,301 E Nine Road,Pensacola,32514,Escambia,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"301 E Nine Road, Pensacola, FL 32514",30.532395,-87.262019
144,FLUENT,2090 U.S. 98,Mary Esther,32569,Okaloosa,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"2090 U.S. 98, Mary Esther, FL 32569",30.412156,-86.744638
145,FLUENT,3460 US Hwy 441,Fruitland Park,34731,Lake,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"3460 US Hwy 441, Fruitland Park, FL 34731",28.844858,-81.898894
146,FLUENT,622 S Federal Highway,Deerfield Beach,33441,Broward,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"622 S Federal Highway, Deerfield Beach, FL 33441",26.309080,-80.092070
147,FLUENT,9637 US Hwy 98,Miramar Beach,32550,Walton,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"9637 US Hwy 98, Miramar Beach, FL 32550",30.377613,-86.336454
148,FLUENT,2620 S US Hwy 1,Fort Pierce,34982,St. Lucie,833-735-8368,info@getfluent.com,Dispensing Authorization,MMTC-2015-0003,"2620 S US Hwy 1, Fort Pierce, FL 34982",27.420541,-80.325950


In [42]:

#radius in miles
radius = 5
results_dict = {'ADDRESS':[], 'competitors':[], 'Latitude': [], 'Longitude': []}
competitor_dict = {'FLUENT Dispensary': [], 'Competitor List': []}

for fluent_dispensary in fluent_dispensaries.iterrows():

    number_of_competitors = 0
    
    for competitor in merged_df_with_coordinates.iterrows():

        fluent_coordinates = [fluent_dispensary[1]['Latitude'], fluent_dispensary[1]['Longitude']]
        competitor_coordinates = [competitor[1]['Latitude'], competitor[1]['Longitude']]

        distance_to_competitor = find_distance(fluent_coordinates, competitor_coordinates)

        if distance_to_competitor <= radius and distance_to_competitor != 0:
            number_of_competitors += 1

            competitor_dict['FLUENT Dispensary'].append(fluent_dispensary[1]['Full Address'])
            competitor_dict['Competitor List'].append(competitor[1]['Full Address'])
    
    results_dict['ADDRESS'].append(fluent_dispensary[1]['ADDRESS'])
    results_dict['competitors'].append(number_of_competitors)
    results_dict['Latitude'].append(fluent_dispensary[1]['Latitude'])
    results_dict['Longitude'].append(fluent_dispensary[1]['Longitude'])

results_dict


{'ADDRESS': ['971 Cassat Avenue',
  '2385 S Ferdon Boulevard',
  '500 E Chase Street',
  '763 Atlantic Boulevard',
  '301 E Nine Road',
  '2090 U.S. 98',
  '3460 US Hwy 441',
  '622 S Federal Highway',
  '9637 US Hwy 98',
  '2620 S US Hwy 1',
  '9611 N Kendall Drive',
  '5827 SW 40th Street',
  '1406 N University Drive',
  '10607 E Colonial Drive',
  '3611 US Hwy 98',
  '4825 US Hwy 98 N',
  '9510 Atlantic Boulevard',
  '606 S Dale Mabry Highway',
  '103 SW 3rd Place',
  '5138 Deer Park Drive',
  '5611 Six Mile Commercial Court',
  '11245 SW 211 Street',
  '6340 N Wickham Road',
  '487 State Road 436',
  '921 S Missouri Avenue',
  '175 NW 167th Street',
  '601 34th Street N',
  '1901 N Orange Avenue',
  '1 S Dixie Highway',
  '3400 SW 34th Street',
  '109 Racetrack Road NE',
  '9901 San Jose Boulevard',
  '1902 Thomasville Road'],
 'competitors': [8,
  1,
  8,
  6,
  7,
  0,
  3,
  9,
  1,
  5,
  16,
  11,
  4,
  14,
  5,
  5,
  5,
  14,
  5,
  3,
  6,
  2,
  1,
  11,
  16,
  11,
  13,

In [43]:
results = pd.DataFrame(results_dict)

# cleaning
results['ADDRESS'] = results['ADDRESS'].str.upper()
results['Latitude'] = results['Latitude'].astype(float).round(3)
results['Longitude'] = results['Longitude'].astype(float).round(3)
results['results_address'] = results['ADDRESS']

results_export_df = results[['results_address', 'competitors']]

# export
results_export_df.to_csv('./exports/dataframe_exports/results.csv', index=False)

results_export_df

,results_address,competitors
0,971 CASSAT AVENUE,8
1,2385 S FERDON BOULEVARD,1
2,500 E CHASE STREET,8
3,763 ATLANTIC BOULEVARD,6
4,301 E NINE ROAD,7
5,2090 U.S. 98,0
6,3460 US HWY 441,3
7,622 S FEDERAL HIGHWAY,9
8,9637 US HWY 98,1
9,2620 S US HWY 1,5


In [44]:
competitor_dict_df = pd.DataFrame(competitor_dict)
competitor_dict_df.rename(columns={'FLUENT Dispensary': 'results_address', 'Competitor List': 'competitor_address'}, inplace=True)
competitor_dict_df.to_csv('./exports/dataframe_exports/competitor_list.csv', index=False)

In [46]:
# import location masterlist
masterlist_df = pd.read_csv('./exports/dataframe_exports/Retail Location Master List - For Competitor Counts.csv')
masterlist_df.head()

,location_id,location_name,region
0,30,Cape Coral,Western
1,26,Casselberry,Central
2,23,Clearwater,Western
3,37,Coral Gables,Southern
4,41,Coral Springs,Southern
